In [1]:
#import gym_duckietown
import matplotlib.pyplot as plt
import numpy as np
import logging
#from gym_duckietown.simulator import Simulator
from casadi import *
import sys
sys.path.insert(0,'..')
import utils
from utils import get_trajectory, get_position, my_odometry

# Hyperparameters
gym_duckietown.logger.setLevel(logging.WARNING)
plt.rcParams['figure.figsize'] = [7, 7]
test_results = True
T = 15 # Simulation time
N = 10 # MPC horizon
n_from_index = 0 # How distant the first point is
frame_rate = 10

import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size'] = 25
#matplotlib.rcParams['axes.xmargin'] = 0

In [ ]:
env = Simulator("ETH_large_loop",
    full_transparency=True,
    domain_rand=False,
    user_tile_start=[3,1],
    seed=42,
    max_steps=float("inf"),
    robot_speed=1.0,
    frame_rate=frame_rate
)
#env.unwrapped.start_pose = [[0.11699990272521976, 0, 0.41029359288296474], pi/2]
env.unwrapped.start_pose = [[0.3, 0, 0.41029359288296474], pi/2]
dt = 1.0 / env.frame_rate
env.reset()
traj = get_trajectory(env, samples=200, method="distance", last_value_distance=1.01)
traj = traj + np.array([-1.25, -1.5])
inside, outside = utils.get_border(traj, distance=0.2)
x_min = traj[:,0].min()
y_min = traj[:,1].min()
#fig = plt.figure(constrained_layout=True, figsize=(10, 5))
#ax_dict = fig.subplot_mosaic("""AB""")
#ax_dict["A"].plot(*inside.T, "g--")
#ax_dict["A"].plot(*outside.T, "g--")
#utils.show_on_map_array(env, inside - np.array([-1.25, -1.5]), fig=ax_dict["B"])
#utils.show_on_map_array(env, outside- np.array([-1.25, -1.5]), fig=ax_dict["B"])
pose = get_position(env)
pose.x = 0.79
pose.y = 0.0
pose.theta = pi/2

In [ ]:
from scipy import spatial
from matplotlib.legend_handler import HandlerPatch
import matplotlib.patches as mpatches

def make_legend_arrow(legend, orig_handle,
                      xdescent, ydescent,
                      width, height, fontsize):
    p = mpatches.FancyArrow(0, 0.5*height, width, 0, length_includes_head=True, head_width=0.75*height)
    return p

distance,index = spatial.KDTree(traj).query([pose.x, pose.y])
pose.x = traj[index,0]
pose.y = traj[index,1]
index += n_from_index

f"Samples must be at least: {np.round(np.sum( np.sqrt(np.sum( np.diff(traj, axis=0)**2, axis=1 )) ) / (0.6*dt))}"
#utils.show_on_map_array(env, traj[:,:2] - np.array([-1.25, -1.5]))
# Compute the angle between every consecutive points in traj
angles = np.zeros(traj.shape[0])
angles[:-1] = np.arctan2(traj[1:,1]-traj[:-1,1], traj[1:,0]-traj[:-1,0])
angles[-1] = np.arctan2(traj[0,1]-traj[-1,1], traj[0,0]-traj[-1,0])
angles[angles < 0] += 2 * np.pi
#idx = []
#for i in range(len(angles)-1):
#    if mod((angles[i] - angles[i+1]),pi) >= pi/2:
#        idx.append(i)
# Add angles to traj
traj = np.concatenate((traj, angles.reshape(-1,1)), axis=1)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,9), dpi=100)
for x, y, t in traj[::3]:
    dx = 0.03*np.cos(t)
    arrow = ax.arrow(x, y, dx, 0.03*np.sin(t), color='g', width=0.01,head_width=0.04, zorder=99)
#ax.plot(*traj[:,:2].T, "k")
ax.plot(*inside.T, "k--", label='inside reference: $r_{ins,k}$', lw=1.25)
ax.plot(*outside.T, "k-.", label='outside reference: $r_{out,k}$', lw=1.25)
ax.arrow(pose.x, pose.y, 0.07*np.cos(pose.theta), 0.07*np.sin(pose.theta), color='r', head_width=0.06,  zorder=100, label=r'start pose ${p}_0$')

handles, labels = ax.get_legend_handles_labels()
handles.append(arrow)
labels.append('reference $r_k$')
#specify order of items in legend
order = [3,0,1,2]

ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order], handler_map={mpatches.FancyArrow : HandlerPatch(patch_func=make_legend_arrow)}, fontsize=15, loc='best')
#ax.legend(handles, labels, handler_map={mpatches.FancyArrow : HandlerPatch(patch_func=make_legend_arrow)}, fontsize=7, loc='best')

#ax.legend([arrow,], ['reference $r_k$',], handler_map={mpatches.FancyArrow : HandlerPatch(patch_func=make_legend_arrow),
#                    })
#plt.scatter(pose.x, pose.y, c="r")
#for idx in range(0,len(traj),10):
#    plt.scatter(traj[idx,0], traj[idx,1], c="g")
#    plt.scatter(inside[idx,0], inside[idx,1], c="g")
#    plt.scatter(outside[idx,0], outside[idx,1], c="g")
#ax.set_xlim([-2,1.1])
ax.margins(x=0.1)
ax.set_xlabel('$x$ coordinate [$m$]')
ax.set_ylabel('$y$ coordinate [$m$]')
plt.grid(True)
plt.gca().set_aspect('equal')
plt.savefig('map.eps', format='eps')
plt.show()

In [ ]:
index = DM(index)
traj = DM(traj)

In [ ]:
up = 5
wp = 4
# parameters for forced dynamics
u_alpha_r = 1.5
u_alpha_l = 1.5
w_alpha_r = 15  # modify this for trim
w_alpha_l = 15
# States
x0 = MX.sym('x')
y0 = MX.sym('y')
th0 = MX.sym('th')
v0 = MX.sym('v')
w0 = MX.sym('w')
x = vertcat(x0, y0, th0, v0, w0) # Always vertically concatenate the states --> [n_x,1]
# Inputs
wl = MX.sym('wl')
wr = MX.sym('wr')
u = vertcat(wl, wr) # Always vertically concatenate the inputs --> [n_u,1]
# System dynamics (CAN BE NONLINEAR! ;))
# x_long_dot_dot = -u1*v0 + u_alpha_r*wr + u_alpha_l*wl
# w_dot_dot = -w1*w0 + w_alpha_r*wr - w_alpha_l*wl
v1 = (1-up*dt)*v0 + u_alpha_r*dt*wr + u_alpha_l*dt*wl
w1 = (1-wp*dt)*w0 + w_alpha_r*dt*wr - w_alpha_l*dt*wl
x1 = x0 + v0*dt*np.cos(th0 + w0*dt/2)
y1 = y0 + v0*dt*np.sin(th0 + w0*dt/2)
# Cannot use atan2 because x1 and y1 are approximated while th1 is not
theta1 = th0 + w0*dt
dae = vertcat(x1, y1, theta1, v1, w1)
F = Function('F',[x,u],[dae],['x','u'],['dae'])
if test_results:
    print(F([pose.x, pose.y, pose.theta, 0, 0], [1, 1]))
N_test=20
# Test the problem definition
sim = F.mapaccum(N_test) # To call the F function N times
x0 = [pose.x, pose.y, pose.theta, 0, 0]

res = np.array(sim(x0, [1, 0.2]))
fig = plt.figure()
tgrid = np.linspace(0,T,N)
# plt.plot(tgrid, np.column_stack((x0, res)).T)
plt.plot(res[0, :], res[1, :], 'r*')
env.reset()
for action in [[1, 0.2]]*(N_test+3):
    obs, reward, done, info = env.step(action)
    p = get_position(env)
    plt.scatter(p.x, p.y, c='b')
_ = env.reset()

Initialize states, inputs, and system dynamic equation

In [ ]:
up = 5
wp = 4
# parameters for forced dynamics
u_alpha_r = 1.5
u_alpha_l = 1.5
w_alpha_r = 15  # modify this for trim
w_alpha_l = 15
# States
x0 = MX.sym('x')
y0 = MX.sym('y')
th0 = MX.sym('th')
v0 = MX.sym('v')
w0 = MX.sym('w')
x = vertcat(x0, y0, th0, v0, w0) # Always vertically concatenate the states --> [n_x,1]
# Inputs
wl = MX.sym('wl')
wr = MX.sym('wr')
u = vertcat(wl, wr) # Always vertically concatenate the inputs --> [n_u,1]
# System dynamics (CAN BE NONLINEAR! ;))
# x_long_dot_dot = -u1*v0 + u_alpha_r*wr + u_alpha_l*wl
# w_dot_dot = -w1*w0 + w_alpha_r*wr - w_alpha_l*wl
v1 = (1-up*dt)*v0 + u_alpha_r*dt*wr + u_alpha_l*dt*wl
w1 = (1-wp*dt)*w0 + w_alpha_r*dt*wr - w_alpha_l*dt*wl
x1 = x0 + v0*dt*np.cos(th0 + w0*dt/2)
y1 = y0 + v0*dt*np.sin(th0 + w0*dt/2)
# Cannot use atan2 because x1 and y1 are approximated while th1 is not
theta1 = th0 + w0*dt
dae = vertcat(x1, y1, theta1, v1, w1)
F = Function('F',[x,u],[dae],['x','u'],['dae'])
if test_results:
    print(F([pose.x, pose.y, pose.theta, 0, 0], [1, 1]))
N_test=20
# Test the problem definition
sim = F.mapaccum(N_test) # To call the F function N times
x0 = [pose.x, pose.y, pose.theta, 0, 0]

res = np.array(sim(x0, [1, 0.2]))
fig = plt.figure()
tgrid = np.linspace(0,T,N)
# plt.plot(tgrid, np.column_stack((x0, res)).T)
plt.plot(res[0, :], res[1, :], 'r*')
env.reset()
for action in [[1, 0.2]]*(N_test+3):
    obs, reward, done, info = env.step(action)
    p = get_position(env)
    plt.scatter(p.x, p.y, c='b')
_ = env.reset()

In [ ]:
delay = round(0.15/dt)
u_delay0 = DM(np.zeros((2, delay)))
# N += delay

# Initial reference point
r0 = repmat(traj[index, :2].T, 1, N+1)
# Initial reference angles
t0 = repmat(traj[index, 2].T, 1, N+1)

# With preview
# Initial reference point
r0 = traj[index:index+N+1, :2].T
t0 = traj[index:index+N+1, 2].T
ins0 = DM(inside)[index:index+N+1, :].T
out0 = DM(outside)[index:index+N+1, :].T

In [ ]:
#weights_0 = [1e3, 1e2, 1e-2, 1]
weights_0 = [1, .5, 50, 0, 0, 5]
theta_0 = [0, 0, 0, 0]
gamma = 0.9
u_prev_0 = [.0, .0]

In [ ]:
def position_cost(x,r):
    delta = 10
    return delta**2 * (np.sqrt(1 + (((x[0]-r[0])**2 + (x[1]-r[1])**2)/delta**2))-1)
def heading_cost(psi,tr):
    return (1 - np.cos(psi - tr))/2

In [ ]:
opti = Opti()
######################## VARIABLE/PARAMETERS ############################################################
x = opti.variable(5,N+1)            # States with dimension (n_x, prediction_horizon+1) (x(0),...,x(N+1))
u = opti.variable(2,N)#-delay)      # Inputs with dimension (n_u, prediction_horizon) (u(0),...,u(N))
slack1 = opti.variable(1,N)         # slack variable 1
slack2 = opti.variable(1,N)         # slack variable 2
p = opti.parameter(5,1)             # Parameter to set the initial state conditions (x(0))
r = opti.parameter(2,N+1)           # Parameter to set the reference position (r(0),...,r(N+1))
tr = opti.parameter(1,N+1)          # Parameter to set the reference angles
ins = opti.parameter(2,N+1)         # Parameter to set the inside boundary
out = opti.parameter(2,N+1)         # Parameter to set the outside boundary
u_delay = opti.parameter(2,delay)   # Parameter to set the delay (NOT USED)
u_prev = opti.parameter(2,1)        # Parameter to set previus input value (u(-1))
weights = opti.parameter(6)         # Parameter for the cost function weights  
theta = opti.parameter(4)           # Additional learned parameters
######################## COST FUNCTION WEIGHTS ############################################################
Q1 = weights[0]
Q2 = weights[1]
Q3 = weights[2]
Q4 = weights[3]
Q5 = weights[4]
R = weights[5]
theta_lambda = theta[0]
theta_f_1 = theta[1]
theta_f_2 = theta[2]
theta_f_3 = theta[3]
######################## INITIAL VALUES ############################################################
cost = 1/(x[3,0]+1e-6)**2 + 1e1*slack1[:,0] + 1e1*slack2[:,0]

opti.subject_to(x[:,1]==F(x[:,0],u[:,0]))         # system dynamics (k=0)
opti.subject_to(x[:,0] == p)                      # set initial x_0
opti.subject_to((sumsqr(x[:2,0])) + slack1[:,0] >= (sumsqr(ins[:,0])))
opti.subject_to((sumsqr(x[:2,0])) - slack2[:,0] <= (sumsqr(out[:,0])))
######################## STAGE COST/RECURSIVE CONTRAINTS ############################################################
for k in range(1,N):
    cost += gamma**k * (1/(x[3, k]+1e-6)**2 + 1e1*slack1[:,k] + 1e1*slack2[:,k])
    opti.subject_to(x[:,k+1]==F(x[:,k],u[:,k]))
    opti.subject_to((sumsqr(x[:2,k])) + slack1[:,k] >= (sumsqr(ins[:,k])))
    opti.subject_to((sumsqr(x[:2,k])) - slack2[:,k] <= (sumsqr(out[:,k])))
######################## TERMINAL COST/CONSTRAINTS ############################################################
cost += gamma**N * (theta_f_1*sumsqr(x[:2, N]-r[:,N])+ theta_f_2*heading_cost(x[2, k],tr[:,k]) + theta_f_3*sumsqr(x[3, k]-0.6))
######################## ADDITIONAL PARAMETERS ############################################################
opti.subject_to(vec(u) <= 1)               # actuator LB
opti.subject_to(vec(u) >= -1)              # actuator LB
opti.subject_to(x[3,:] >= 0)               # Enforce forward movement
opti.subject_to(vec(slack1) >= 0)          # slack 1 condition
opti.subject_to(vec(slack2) >= 0)          # slack 2 condition
######################## INITIALIZE PARAMETERS ############################################################
opti.set_value(p,[pose.x, pose.y, pose.theta, 0, 0]) # Set the initial x0 value
opti.set_value(r,r0) # Set the initial reference points
opti.set_value(tr,t0) # Set the initial reference angles
opti.set_value(ins,ins0) # Set the initial reference angles
opti.set_value(out,out0) # Set the initial reference angles
opti.set_value(u_delay, u_delay0) # Initial delay
opti.set_value(u_prev, u_prev_0) # Initial delay
opti.set_value(weights, weights_0)
opti.set_value(theta, theta_0) 
######################## SOLVER SPECIFICATIONS ############################################################
# Ipopt
opts = dict()
opts["ipopt.print_level"] = 0
opts["print_time"] = False
opti.solver('ipopt',opts)
opti.minimize(cost)
######################## CREATE VALUE FUNCTION V(s) ############################################################
V = opti.to_function('V',[p, r, tr, ins, out, u_delay, u_prev, weights, theta], # inputs
                         [cost, u[:,0], gradient(cost + opti.lam_g.T @ opti.g, weights), gradient(cost + opti.lam_g.T @ opti.g, theta)], # outputs
                         ['p', 'r', 'tr', 'ins', 'out','u_delay', 'u_prev', 'weights', 'theta'],
                         ['cost', 'u_opt', 'gradient_weights', 'gradient_theta'])
######################## TESTING ############################################################
# Test over 1 time horizon
if test_results:
    sol = opti.solve()

In [ ]:
u_s = [.0, .0]
opti = Opti()

x = opti.variable(5,N+1)    # States with dimension (n_x, prediction_horizon+1) (x(0),...,x(N+1))
u = opti.variable(2,N)#-delay)      # Inputs with dimension (n_u, prediction_horizon) (u(0),...,u(N))
slack1 = opti.variable(1,N)
slack2 = opti.variable(1,N)
p = opti.parameter(5,1)     # Parameter to set the initial state conditions (x(0))
r = opti.parameter(2,N+1)   # Parameter to set the initial index
tr = opti.parameter(1,N+1)   # Parameter to set the initial reference angles
ins = opti.parameter(2,N+1)   # Inside reference
out = opti.parameter(2,N+1)   # Outside reference
u_delay = opti.parameter(2,delay) # Parameter to set the delay
u_prev = opti.parameter(2,1)
u_0 = opti.parameter(2)

weights = opti.parameter(6)
theta = opti.parameter(4)
#beta = opti.parameter(1)

#beta1 = beta[0]
#beta2 = beta[1]

Q1 = weights[0]
Q2 = weights[1]
Q3 = weights[2]
Q4 = weights[3]
Q5 = weights[4]
R = weights[5]

theta_lambda = theta[0]
theta_f_1 = theta[1]
theta_f_2 = theta[2]
theta_f_3 = theta[3]

# With delay:
# Q, Q2, Q3, R = 100, 5e-4, 1, 1e-3
#obj = Q*sumsqr(x[:2, :]-r) + Q2*sumsqr(mod(x[2, :]-tr+pi, 2*pi)-pi) + Q3*sumsqr(x[3, :]-0.6) + R*sumsqr(u)#R*sumsqr(u[:,1:]-u[:,:-1])
# atan method is faster than mod (20s vs 23s on 1000 iterations) but it allows car to go backward
# obj = Q*sumsqr(x[:2, :]-r) + Q2*sumsqr(atan2(sin(x[2, :]-tr), cos(x[2, :]-tr))) + R*sumsqr(u)

        # Setup the problem 

# Constraints Definition
# Constraints Definition
cost = 1/(x[3,0]+1e-6)**2 + 1e1*slack1[:,0] + 1e1*slack2[:,0]
opti.subject_to((sumsqr(x[:2,0])) + slack1[:,0] >= (sumsqr(ins[:,0])))
opti.subject_to((sumsqr(x[:2,0])) - slack2[:,0] <= (sumsqr(out[:,0])))
# 1) System dynamics constraint over the prediction horizon
for k in range(1,N):
    cost += gamma**k * (1/(x[3, k]+1e-6)**2 + 1e1*slack1[:,k] + 1e1*slack2[:,k])
    opti.subject_to(x[:,k+1]==F(x[:,k],u[:,k]))
    opti.subject_to((sumsqr(x[:2,k])) + slack1[:,k] >= (sumsqr(ins[:,k])))
    opti.subject_to((sumsqr(x[:2,k])) - slack2[:,k] <= (sumsqr(out[:,k])))   

cost += gamma**N * (theta_f_1*sumsqr(x[:2, N]-r[:,N])+ theta_f_2*heading_cost(x[2, k],tr[:,k]) + theta_f_3*sumsqr(x[3, k]-0.6))

opti.subject_to(x[:,1]==F(x[:,0],u[:,0]))
# 2) Additional states/inputs constraints 
opti.subject_to(vec(u) <= 1)
opti.subject_to(vec(u) >= -1)
#opti.subject_to(sumsqr(x[:2,:]) >= sumsqr(r[:,:]))
opti.subject_to(x[3,:] >= 0)
# 3) Force the state variable at instant 0 to be equal to the parameter value containing x0 value
opti.subject_to(x[:,0] == p)
opti.subject_to(u[:,0] == u_0)
opti.subject_to(vec(slack1) >= 0)
opti.subject_to(vec(slack2) >= 0)

opti.set_value(p,[pose.x, pose.y, pose.theta, 0, 0]) # Set the initial x0 value
opti.set_value(r,r0) # Set the initial reference points
opti.set_value(tr,t0) # Set the initial reference angles
opti.set_value(ins,ins0) # Set the initial reference angles
opti.set_value(out,out0) # Set the initial reference angles
opti.set_value(u_delay, u_delay0) # Initial delay
opti.set_value(u_prev, u_prev_0) # Initial delay
opti.set_value(u_0, u_s)

opti.set_value(weights, weights_0)
opti.set_value(theta, theta_0)

opti.minimize(cost)

# Ipopt
opts = dict()
opts["ipopt.print_level"] = 0
opts["print_time"] = False
opti.solver('ipopt',opts)

# Test over 1 time horizon
    
Q = opti.to_function('Q',[p, r, tr, ins, out, u_delay, u_prev, u_0, weights, theta], # inputs
                         [cost, gradient(cost + opti.lam_g.T @ opti.g, weights), gradient(cost + opti.lam_g.T @ opti.g, theta)], # outputs
                         ['p', 'r', 'tr', 'ins', 'out', 'u_delay', 'u_prev', 'u_0', 'weights', 'theta'],
                         ['cost', 'gradient_weigths', 'gradient_theta'])

if test_results:
    sol = opti.solve()

In [ ]:
from skspatial.objects import Line
from IPython import display as ipythondisplay
from datetime import datetime

def stage_cost(x, r, tr, x_next, x_prev):
    return 1/(x[3]+1e-6)**2 + np.maximum(0, 1e1*float((sumsqr(ins[:,0])) - (sumsqr(x[:2])))) + np.maximum(0, 1e1*float((sumsqr(x[:2])) - (sumsqr(out[:,0]))))

n_batch = 100
# MPC loop
gamma = 0.9
weights_0 = [0, 0, 0, 0, 0, 0]
theta_0 = [0, 0, 0, 0]


lap_dict = dict()

X_log = np.empty((5,0))
U_log = np.empty((2,0))

x = [pose.x, pose.y, pose.theta, 0, 0]
start_angle = np.arctan2(x[1], x[0])
start_angle = start_angle if start_angle >= 0 else start_angle + 2*np.pi


x_prev = x
r = r0
tr = t0
ins = ins0
out = out0
idx = index
u_delay = u_delay0
u_prev = u_prev_0
errors = []
error_x = []
error_y = []
error_theta = []
it_vect = []
weights_vect = []
theta_vect = []
delta_vect = []
grad_weights_vect = []
grad_theta_vect = []
stage_cost_vect = []
time_vect = []

kdtree = spatial.KDTree(traj[:, :2])

_,start_idx = kdtree.query(np.array([x[0], x[1]]).reshape(-1))

x_prev = x
i_prev = 0

t = 0
flag = False

autotune = False


for i in range(10000000):

    start = datetime.now()
    V_cost, a, _, _ = V(x, r, tr, ins, out, u_delay, u_prev, weights_0, theta_0)
    time_vect.append((datetime.now() - start).total_seconds()*1000)

    U_log = np.column_stack((U_log, a))
    X_log = np.column_stack((X_log, x))
    
    # simulate
    x_next = F(x,a)    
    
    Q_cost, Q_grad_weights, Q_grad_theta = Q(x, r, tr, ins, out, u_delay, u_prev, a, weights_0, theta_0)

    l = stage_cost(x, r, tr, x_next, x_prev)
    delta = l + gamma * V(x_next, r, tr, ins, out, u_delay, u_prev, weights_0, theta_0)[0] - Q_cost

    x_prev = x
    u_prev = a
    x = x_next
    
    distance = (traj[idx, 0] - x[0])**2 + (traj[idx, 1] - x[1])**2
    it_vect.append(i)
    errors.append(np.sqrt(distance))
    error_x.append(traj[idx, 0] - x[0])
    error_y.append(traj[idx, 1] - x[1])
    error_theta.append(mod(x[2],2*np.pi)-traj[idx, 2])
    weights_vect.append(DM(weights_0))
    theta_vect.append(DM(theta_0))
    delta_vect.append(delta)
    grad_weights_vect.append(Q_grad_weights)
    grad_theta_vect.append(Q_grad_theta)
    stage_cost_vect.append(l)
    
    _,idx = kdtree.query(np.array([x[0], x[1]]).reshape(-1))
    
    current_angle = np.arctan2(x[1], x[0])
    current_angle = current_angle if current_angle >= 0 else current_angle + 2*np.pi

    if i > 5 and (float(current_angle) - start_angle >= - 0.05 and float(current_angle) - start_angle <= 0.05):
        print('Lap %i'%(len(lap_dict)+1))
        lap_dict[len(lap_dict)+1] = [it_vect[i_prev:], X_log[:,i_prev:], U_log[:,i_prev:], delta_vect[i_prev:], weights_vect[i_prev:], theta_vect[i_prev:], stage_cost_vect[i_prev:]]
        i_prev = i
    
    if idx+N+1 < traj.shape[0]:
        r = traj[idx:idx+N+1, :2].T
        tr = traj[idx:idx+N+1, 2].T
        ins = DM(inside)[idx:idx+N+1, :].T
        out = DM(outside)[idx:idx+N+1, :].T
    else:
        r = vertcat(traj[idx:, :2], traj[:idx+N+1-traj.shape[0], :2]).T
        tr = vertcat(traj[idx:, 2], traj[:idx+N+1-traj.shape[0], 2]).T
        ins = vertcat(DM(inside)[idx:, :], DM(inside)[:idx+N+1-inside.shape[0], :]).T
        out = vertcat(DM(outside)[idx:, :], DM(outside)[:idx+N+1-inside.shape[0], :]).T
        
    #print((datetime.now() - start).total_seconds()*1000)

    if len(lap_dict)==5: # 30 ok
        break
        #ax.plot(*X_log[:2,:], c='r')
        #ax.text(0,0,"Steps: {iteration} Last time: {time:.2f}s".format(iteration=i, time=X_log.shape[1]*dt))
        #plt.draw()
        #ipythondisplay.clear_output(wait=True)
        #ipythondisplay.display(plt.gcf())
    #r = traj[idx:idx+N+1, :2].T
    #tr = traj[idx:idx+N+1, 2].T

In [ ]:
np.array(time_vect).mean()
np.array(time_vect).std()

In [ ]:
fig = plt.figure(constrained_layout=True, dpi=200)
traj = np.array(traj)
index = int(index)
ax_dict = fig.subplot_mosaic("""A""")
ax_dict["A"].plot(traj[:,0], traj[:,1], "b")
ax_dict["A"].plot(inside[:,0], inside[:,1], "b--")
ax_dict["A"].plot(outside[:,0], outside[:,1], "b--")
ax_dict["A"].text(X_log[0,0], X_log[1,0], 'START')

for i, (key, val) in enumerate(lap_dict.items()):
    #print(val[1].shape)
    if i%1==0:
        ax_dict["A"].plot(val[1][0,:], val[1][1,:], label='Lap n. %i'%(i+1))
plt.legend()
plt.grid(True)
plt.gca().set_aspect('equal')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10,5), dpi=200)

ax1.plot(np.array(grad_weights_vect).reshape(-1,6)[:,0], label='$Q_1$')
ax1.plot(np.array(grad_weights_vect).reshape(-1,6)[:,1], label='$Q_2$')
ax1.plot(np.array(grad_weights_vect).reshape(-1,6)[:,2], label='$Q_3$')
#ax1.plot(np.array(grad_weights_vect).reshape(-1,6)[:,3], label='$Q_4$')
#ax1.plot(np.array(grad_weights_vect).reshape(-1,6)[:,4], label='$Q_5$')
ax1.plot(np.array(grad_weights_vect).reshape(-1,6)[:,5], label='$R$')
ax1.legend()

ax2.plot(np.array(grad_theta_vect).reshape(-1,4)[:,0], label=r'$\theta_{\lambda}$')
ax2.plot(np.array(grad_theta_vect).reshape(-1,4)[:,1], label=r'$\theta_{V,1}$')
ax2.plot(np.array(grad_theta_vect).reshape(-1,4)[:,2], label=r'$\theta_{V,2}$')
ax2.plot(np.array(grad_theta_vect).reshape(-1,4)[:,3], label=r'$\theta_{V,3}$')
ax2.legend()

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
import pickle as pkl
with open('vars/logs_economic_5_laps.pickle', 'wb') as handle:
    pkl.dump(lap_dict, handle, protocol=pkl.HIGHEST_PROTOCOL)

In [ ]:
plt.plot(moving_average(np.array(stage_cost_vect[10:]).flatten(), 1))

In [ ]:
(1/0.6)**2

In [ ]:
print('weights_vect: ', weights_vect[-1])
print('theta_vect: ', theta_vect[-1])

Con offset costante su x = 0.005, a partire da beta = 0.0 converge a 0.010 (10000 steps)
Con offset costante su x = 0.01, a partire da beta = 0.0 converge a ?? (10000 steps)
Con offset costante su x = 0.005, a partire da beta = 0.01 converge a ?? (10000 steps)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,constrained_layout=False, figsize=(10, 4), dpi=200)
ax1.plot(np.array(theta_vect).reshape(-1,4)[:,0], label=r'$\theta_{\lambda}$')
ax1.plot(np.array(theta_vect).reshape(-1,4)[:,1], label=r'$\theta_{V,1}$')
ax1.plot(np.array(theta_vect).reshape(-1,4)[:,2], label=r'$\theta_{V,2}$')
ax1.plot(np.array(theta_vect).reshape(-1,4)[:,3], label=r'$\theta_{V,3}$')
ax1.legend()
ax1.set_xlabel('steps')
ax1.set_ylabel(r'$\theta$')
ax1.grid(True)

ax2.plot(moving_average(np.array(delta_vect[30:]).flatten(), 500), label='TD error')
ax2.legend()
#ax2.plot(np.array(delta_vect).flatten())
ax2.grid(True)
ax2.set_xlabel('steps')
ax2.set_ylabel(r'$\delta$')
plt.tight_layout()

In [ ]:
weights_vect[-1]

In [ ]:
f"Max reached speed: {X_log[3,:].max()}, max speed is 0.6"

In [ ]:
np.mean(X_log[3,:])

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10,5), dpi=200)
ax1.plot(moving_average(X_log[3,:],1))
ax1.axhline(np.mean(X_log[3,:]), c='r', label='mean value')
ax1.fill_between(np.arange(500), np.mean(X_log[3,:])-np.std(X_log[3,:]),np.mean(X_log[3,:])+np.std(X_log[3,:]), color='r', alpha=0.1)
ax1.grid(True)
ax1.set_xlabel('steps')
ax1.set_ylabel('speed [m/s]')
ax1.set_ylim(0,0.6)
ax1.margins(x=0)
ax1.legend()

ax2.plot(moving_average(X_log[4,:],1))
ax2.axhline(np.mean(X_log[4,:]), c='r', label='mean value')
ax2.fill_between(np.arange(500), np.mean(X_log[4,:])-np.std(X_log[4,:]),np.mean(X_log[4,:])+np.std(X_log[4,:]), color='r', alpha=0.1)
ax2.grid(True)
ax2.set_xlabel('steps')
ax2.set_ylabel('ang. speed [rad/s]')
ax2.set_ylim(-np.pi,np.pi)
ax2.margins(x=0)
plt.tight_layout()

In [ ]:
errors = np.array(errors).reshape(-1)
f"Mean error: {errors.mean()}"

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(10, 5), dpi=200)
traj = np.array(traj)
index = int(index)
ax_dict = fig.subplot_mosaic("""AC""")
ax_dict["A"].text(X_log[0,0], X_log[1,0], 'START')
ax_dict["A"].plot(traj[:,0], traj[:,1], "b")
ax_dict["A"].plot(inside[:,0], inside[:,1], "b--")
ax_dict["A"].plot(outside[:,0], outside[:,1], "b--")
ax_dict["A"].plot(traj[index,0], traj[index,1], 'go')
ax_dict["A"].plot(X_log[0,:], X_log[1,:], "r")
ax_dict["A"].plot(traj[idx, 0], traj[idx,1], 'yo')
utils.show_on_map_array(env, np.array([X_log[0,:], X_log[1,:]]).T  - np.array([-1.25, -1.5]), fig=ax_dict["C"])

In [ ]:
t*1/10

In [ ]:
print(np.mean(np.abs(curvature_val_observed_tuned_curv).flatten()[np.abs(curvature_val_observed_tuned_curv).flatten() < 0.5]))
print(np.mean(np.abs(curvature_val_observed_tuned_speed).flatten()[np.abs(curvature_val_observed_tuned_speed).flatten() < 0.5]))
print(np.mean(np.abs(curvature_val_observed_non_tuned).flatten()[np.abs(curvature_val_observed_non_tuned).flatten() < 0.5]))

In [ ]:
#print(np.sum(np.abs(curvature_val_observed_tuned_curv)))
#print(np.sum(np.abs(curvature_val_observed_tuned_speed)))
#print(np.sum(np.abs(curvature_val_observed_non_tuned)))
print(np.mean(np.abs(curvature_val_observed_tuned_curv)))
print(np.mean((curvature_val_observed_tuned_speed)))
print(np.mean((curvature_val_observed_non_tuned)))

In [ ]:
#curvature_val_reference = curvature(traj[:,0], traj[:,1])
curvature_val_observed_tuned_curv = curvature(X_log[0,:], X_log[1,:])
#curvature_val_observed_tuned_speed = curvature(X_log[0,:], X_log[1,:])
#curvature_val_observed_non_tuned = curvature(X_log[0,:], X_log[1,:])
#curvature_val = curvature(r[0,:], r[1,:])
#plt.plot(np.array(curvature_val))
plt.figure(figsize=(15,4),dpi=200)
plt.plot(np.abs(np.array(curvature_val_observed_non_tuned)), label='not tuned')
plt.plot(np.abs(np.array(curvature_val_observed_tuned_curv)), label='tuned - curvature')
plt.plot(np.abs(np.array(curvature_val_observed_tuned_speed)), label='tuned - speed')
plt.margins(x=0)
plt.grid(True)
plt.legend()
plt.xlabel('steps')
plt.ylabel('curvature')

In [ ]:
plt.figure(figsize=(10,5), dpi=200)
#plt.xticks(np.arange(0, len(errors)*dt+1, 1.0))
plt.plot(moving_average(np.sqrt(errors), 5))
plt.axhline(y=0.025, color='r', linestyle='-')
plt.xlabel("steps")
plt.ylabel("Distance from reference (m)")
plt.grid(True)

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(16, 10))
ax_dict = fig.subplot_mosaic("""A
        B
        C""")
error_x = np.array(error_x).reshape(-1)
ax_dict["A"].plot(np.arange(len(error_x))*dt, np.abs(error_x))
ax_dict["A"].set_title("Error in x")
ax_dict["A"].set_ylabel("Distance from x (m)")
error_y = np.array(error_y).reshape(-1)
ax_dict["B"].plot(np.arange(len(error_y))*dt, np.abs(error_y))
ax_dict["B"].set_title("Error in y")
ax_dict["B"].set_ylabel("Distance from y (m)")
error_theta_scaled = np.rad2deg(np.array(error_theta).reshape(-1))
ax_dict["C"].plot(np.arange(len(error_theta_scaled))*dt, np.abs(error_theta_scaled))
ax_dict["C"].set_title("Error in theta")
ax_dict["C"].set_ylabel("Distance from theta (deg)")
ax_dict["C"].set_xlabel("Time (s)")

## Parameters sensitivity

In [ ]:
td_graph = False

### Multi parameter sensitivity

In [ ]:
if td_graph:
    errors_Q = []

    for Q_ in np.logspace(-4, 2, 20):
        errors_Q2 = []
        for Q2_ in np.logspace(-4, 2, 20):
            X_log = np.empty((5,0))
            U_log = np.empty((2,0))

            x = [pose.x, pose.y, pose.theta, 0, 0]
            r = r0
            tr = t0
            idx = index
            u_delay = u_delay0
            errors = []
            error_x = []
            error_y = []
            error_theta = []

            kdtree = spatial.KDTree(traj[:, :2])
            for i in range(50):
                u = M(x, r, tr, u_delay, Q_, Q2_, 1, 1)

                U_log = np.column_stack((U_log, u))
                X_log = np.column_stack((X_log, x))

                u_delay = np.column_stack((u_delay, u))[:, -delay:]

                # simulate
                x = F(x,u)

                distance = (traj[idx, 0] - x[0])**2 + (traj[idx, 1] - x[1])**2
                errors.append(distance)
                error_x.append(traj[idx, 0] - x[0])
                error_y.append(traj[idx, 1] - x[1])
                error_theta.append(mod(x[2]-traj[idx, 2]+pi, 2*pi)-pi)

                _,idx = kdtree.query(np.array([x[0], x[1]]).reshape(-1))
                r = traj[idx:idx+N+1, :2].T
                tr = traj[idx:idx+N+1, 2].T
            errors_Q2.append(np.array(errors).reshape(-1).sum())
        errors_Q.append(errors_Q2)

In [ ]:
if td_graph:
    import matplotlib.ticker as mticker
    import matplotlib as mpl

    mpl.style.use("default")
    ax = plt.axes(projection='3d')
    ax.plot_wireframe(np.log10(np.logspace(-4, 2, 20)), np.log10(np.logspace(-4, 2, 20)), np.array(errors_Q), label="J")

    # My axis should display 10⁻¹ but you can switch to e-notation 1.00e+01
    def log_tick_formatter(val, pos=None):
        return f"$10^{{{int(val)}}}$"  # remove int() if you don't use MaxNLocator
        #return f"{10**val:.2e}"      # e-Notation

    ax.xaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
    ax.xaxis.set_major_locator(mticker.MaxNLocator(integer=True))

    ax.yaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
    ax.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))

    ax.set_xlabel('Q1', fontsize=15, rotation=0, labelpad=10)
    ax.set_ylabel('Q2', fontsize=15, labelpad=15)
    ax.set_zlabel('J', fontsize=15, rotation=0, labelpad=10)
    ax.yaxis.set_rotate_label(False)

    ax.grid(True)

    ax.set_title('Q1 vs Q2')

#### Sensitivity to Q1/Q2
FIxed Q3 and R

In [ ]:
errors_Q = []

for Q_ in np.logspace(-2, 3, 30):
    X_log = np.empty((5,0))
    U_log = np.empty((2,0))

    x = [pose.x, pose.y, pose.theta, 0, 0]
    r = r0
    tr = t0
    idx = index
    u_delay = u_delay0
    errors = []
    error_x = []
    error_y = []
    error_theta = []

    kdtree = spatial.KDTree(traj[:, :2])
    for i in range(50):
        u = M(x, r, tr, u_delay, Q_, 1, 1e-2, 1)

        U_log = np.column_stack((U_log, u))
        X_log = np.column_stack((X_log, x))

        u_delay = np.column_stack((u_delay, u))[:, -delay:]

        # simulate
        x = F(x,u)

        distance = (traj[idx, 0] - x[0])**2 + (traj[idx, 1] - x[1])**2
        errors.append(distance)
        error_x.append(traj[idx, 0] - x[0])
        error_y.append(traj[idx, 1] - x[1])
        error_theta.append(mod(x[2]-traj[idx, 2]+pi, 2*pi)-pi)

        _,idx = kdtree.query(np.array([x[0], x[1]]).reshape(-1))
        r = traj[idx:idx+N+1, :2].T
        tr = traj[idx:idx+N+1, 2].T
    errors_Q.append(np.array(errors).reshape(-1).sum())

In [ ]:
fig, ax = plt.subplots()
x = np.logspace(-2, 3, 30)
ax.plot(x, errors_Q)
ax.set_xlabel('Q1/Q2', fontsize=15, rotation=0, labelpad=10)
ax.set_ylabel('J', fontsize=15, labelpad=15, rotation=0)
ax.grid()

# Set x scale to logarithmic
ax.set_xscale('log')

### Sensitivity to Q1 vs Q3

In [ ]:
errors_Q2 = []

for Q2_ in np.logspace(-2, 3, 30):
    X_log = np.empty((5,0))
    U_log = np.empty((2,0))

    x = [pose.x, pose.y, pose.theta, 0, 0]
    r = r0
    tr = t0
    idx = index
    u_delay = u_delay0
    errors = []

    kdtree = spatial.KDTree(traj[:, :2])
    for i in range(50):
        u = M(x, r, tr, u_delay, Q2_, 1e-2, 1, 1)

        U_log = np.column_stack((U_log, u))
        X_log = np.column_stack((X_log, x))

        u_delay = np.column_stack((u_delay, u))[:, -delay:]

        # simulate
        x = F(x,u)

        distance = (traj[idx, 0] - x[0])**2 + (traj[idx, 1] - x[1])**2
        errors.append(distance)

        _,idx = kdtree.query(np.array([x[0], x[1]]).reshape(-1))
        r = traj[idx:idx+N+1, :2].T
        tr = traj[idx:idx+N+1, 2].T
    errors_Q2.append(np.array(errors).reshape(-1).sum())

In [ ]:
fig, ax = plt.subplots()
x = np.logspace(-2, 3, 30)
ax.plot(x, errors_Q2)
ax.set_xlabel('Q1/Q3', fontsize=15, rotation=0, labelpad=10)
ax.set_ylabel('J', fontsize=15, labelpad=15, rotation=0)
ax.grid()

# Set x scale to logarithmic
ax.set_xscale('log')

### Sensitivity to Q2/Q3

In [ ]:
errors_Q3 = []

for Q3_ in np.logspace(-5, 5, 40):
    X_log = np.empty((5,0))
    U_log = np.empty((2,0))

    x = [pose.x, pose.y, pose.theta, 0, 0]
    r = r0
    tr = t0
    idx = index
    u_delay = u_delay0
    errors = []

    kdtree = spatial.KDTree(traj[:, :2])
    for i in range(50):
        u = M(x, r, tr, u_delay, 1e2, Q3_, 1, 1)

        U_log = np.column_stack((U_log, u))
        X_log = np.column_stack((X_log, x))

        u_delay = np.column_stack((u_delay, u))[:, -delay:]

        # simulate
        x = F(x,u)

        distance = (traj[idx, 0] - x[0])**2 + (traj[idx, 1] - x[1])**2
        errors.append(distance)

        _,idx = kdtree.query(np.array([x[0], x[1]]).reshape(-1))
        r = traj[idx:idx+N+1, :2].T
        tr = traj[idx:idx+N+1, 2].T
    errors_Q3.append(np.array(errors).reshape(-1).sum())

In [ ]:
fig, ax = plt.subplots()
x = np.logspace(-5, 5, 40)
ax.plot(x, errors_Q3)

ax.set_xlabel('Q2/Q3', fontsize=15, rotation=0, labelpad=10)
ax.set_ylabel('J', fontsize=15, labelpad=15, rotation=0)
ax.grid()

# Set x scale to logarithmic
ax.set_xscale('log')

### Sensitivity to R

In [ ]:
sens_R = False

if sens_R:
    errors_R = []

    for R_ in np.logspace(-4, 2, 20):
        X_log = np.empty((5,0))
        U_log = np.empty((2,0))

        x = [pose.x, pose.y, pose.theta, 0, 0]
        r = r0
        tr = t0
        idx = index
        u_delay = u_delay0
        errors = []

        kdtree = spatial.KDTree(traj[:, :2])
        for i in range(50):
            u = M(x, r, tr, u_delay, 100, 5e-4, 1, R_)

            U_log = np.column_stack((U_log, u))
            X_log = np.column_stack((X_log, x))

            u_delay = np.column_stack((u_delay, u))[:, -delay:]

            # simulate
            x = F(x,u)

            distance = (traj[idx, 0] - x[0])**2 + (traj[idx, 1] - x[1])**2
            errors.append(distance)

            _,idx = kdtree.query(np.array([x[0], x[1]]).reshape(-1))
            r = traj[idx:idx+N+1, :2].T
            tr = traj[idx:idx+N+1, 2].T
        errors_R.append(np.array(errors).reshape(-1).sum())

In [ ]:
if sens_R:
    fig, ax = plt.subplots()
    x = np.logspace(-4, 2, 20)
    ax.plot(x, errors_R)

    # Set x scale to logarithmic
    ax.set_xscale('log')

If you want to save the MPC controller, i.e. the M function for later use, you can do it!

In [ ]:
M.save('M.casadi')

### Gym simulation

The MPC does not consider the delay yet!

In [ ]:
record = False

if record:
    import imageio
    import cv2

    env.reset()
    pose = get_position(env)
    images = []
    positions = []
    odoms = [pose]
    actions = [[0,0]]*10
    v0 = 0
    w0 = 0
    for action in U_log.T:
        env.step(action)
        positions.append(get_position(env))
        img = env.render(mode='rgb_array')
        img = cv2.resize(img, dsize=(400, 300), interpolation=cv2.INTER_CUBIC)
        images.append(img)

        p, v0, w0 = my_odometry(action, odoms[-1].x, odoms[-1].y, odoms[-1].theta, v0, w0, dt=dt)
        odoms.append(p)
        actions.append(action)

    imageio.mimsave("assets/mpc_run.gif", images, format='GIF', fps=env.frame_rate)

In [ ]:
if record:
    plt.plot(*np.array([[p.x, p.y] for p in positions]).T, c='g', label='true', linewidth=5)
    plt.plot(*np.array([[p.x, p.y] for p in odoms]).T, c='r', label='odom')
    plt.legend()

In [ ]:
if record:
    utils.show_on_map(env, positions)

In [ ]:
from IPython.display import HTML, display
# Trick to prevent cache use
display(HTML('<img src="assets/mpc_run.gif?%d" width="750" align="center">' % np.random.randint(10)))